<a href="https://colab.research.google.com/github/philosophynote/Big-Data-Derby/blob/main/20220816.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
class Config:
    name = "baseline-001"
    only_inference = False

    # model_name = "roberta-base"
    # learning_rate = 1e-5
    # max_length = 256
    # epochs = 8
    # batch_size = 16

    # n_fold = 5
    # trn_fold = [0, 1, 2, 3, 4]
    seed = 2022
    # target_col = "target"
    # debug = False

    # Colab Env
    upload_from_colab = True
    
    api_path = "/content/drive/MyDrive/Big-Data-Derb/kaggle.json"
    drive_path = "/content/drive/MyDrive/Big-Data-Derb"
    
    # Kaggle Env
    # kaggle_dataset_path = None

In [2]:
import os
import json
import warnings
import shutil
import logging
import joblib
import random
import datetime
import sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm.auto import tqdm
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

In [3]:
class Logger:

    
    # 参考) https://github.com/ghmagazine/kagglebook/blob/master/ch04-model-interface/code/util.py
    

    def __init__(self, path):
        self.general_logger = logging.getLogger(path)
        stream_handler = logging.StreamHandler()
        file_general_handler = logging.FileHandler(os.path.join(path, 'Experiment.log'))
        if len(self.general_logger.handlers) == 0:
            self.general_logger.addHandler(stream_handler)
            self.general_logger.addHandler(file_general_handler)
            self.general_logger.setLevel(logging.INFO)

    def info(self, message):
        # display time
        self.general_logger.info('[{}] - {}'.format(self.now_string(), message))

    @staticmethod
    def now_string():
        return str(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
    

def seed_everything(seed=42):


    # 参考) https://qiita.com/kaggle_grandmaster-arai-san/items/d59b2fb7142ec7e270a5
    
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)


In [11]:
# Setup
# このコードで Colab 環境なのか Kaggle 環境なのかを判別
COLAB = "google.colab" in sys.modules

if COLAB:
    print("This environment is Google Colab")
    
    # mount
    from google.colab import drive
    if not os.path.isdir("/content/drive"):
        drive.mount('/content/drive') 
	
    # import library
    # ! pip install --quiet transformers
    # ! pip install --quiet iterative-stratification
    # ! pip install --quiet tensorflow-addons

    # use kaggle api (need kaggle token)
    f = open(Config.api_path, 'r')
    json_data = json.load(f) 
    os.environ["KAGGLE_USERNAME"] = json_data["username"]
    os.environ["KAGGLE_KEY"] = json_data["key"]
    
    # set dirs
    DRIVE = Config.drive_path
    EXP = (Config.name if Config.name is not None 
           else get("http://172.28.0.2:9000/api/sessions").json()[0]["name"][:-6])
    INPUT = os.path.join(DRIVE, "Input")
    OUTPUT = os.path.join(DRIVE, "Output")
    SUBMISSION = os.path.join(DRIVE, "Submission")
    OUTPUT_EXP = os.path.join(OUTPUT, EXP) 
    EXP_MODEL = os.path.join(OUTPUT_EXP, "model")
    EXP_FIG = os.path.join(OUTPUT_EXP, "fig")
    EXP_PREDS = os.path.join(OUTPUT_EXP, "preds")

    # make dirs
    for d in [INPUT, SUBMISSION, EXP_MODEL, EXP_FIG, EXP_PREDS]:
        os.makedirs(d, exist_ok=True)

    if not os.path.isfile(os.path.join(INPUT, "big-data-derby-2022.zip")):
        # load dataset
        ! kaggle competitions download -c big-data-derby-2022 -p $INPUT
        ! unzip /content/drive/MyDrive/Big-Data-Derb/Input/big-data-derby-2022.zip -d $INPUT
    
    # utils
    logger = Logger(OUTPUT_EXP)

    # zip解凍
    # shutil.unpack_archive('big-data-derby-2022.zip', INPUT)
  
else:
    print("This environment is Kaggle Kernel")
    
    # set dirs
    INPUT = "../input/big-data-derby-2022"
    EXP, OUTPUT, SUBMISSION = "./", "./", "./"
    EXP_MODEL = os.path.join(EXP, "model")
    EXP_FIG = os.path.join(EXP, "fig")
    EXP_PREDS = os.path.join(EXP, "preds")
    
    # copy dirs
    if Config.kaggle_dataset_path is not None:
        KD_MODEL = os.path.join(Config.kaggle_dataset_path, "model")
        KD_EXP_PREDS = os.path.join(Config.kaggle_dataset_path, "preds")
        shutil.copytree(KD_MODEL, EXP_MODEL)
        shutil.copytree(KD_EXP_PREDS, EXP_PREDS)

    # make dirs
    for d in [EXP_MODEL, EXP_FIG, EXP_PREDS]:
        os.makedirs(d, exist_ok=True)
        
    # utils
    logger = Logger(EXP)

# utils
warnings.filterwarnings("ignore")
sns.set(style='whitegrid')
seed_everything(seed=Config.seed)

This environment is Google Colab
 97% 180M/185M [00:01<00:00, 120MB/s]
100% 185M/185M [00:01<00:00, 131MB/s]
Archive:  /content/drive/MyDrive/Big-Data-Derb/Input/big-data-derby-2022.zip
  inflating: /content/drive/MyDrive/Big-Data-Derb/Input/nyra_2019_complete.csv  
  inflating: /content/drive/MyDrive/Big-Data-Derb/Input/nyra_race_table.csv  
  inflating: /content/drive/MyDrive/Big-Data-Derb/Input/nyra_start_table.csv  
  inflating: /content/drive/MyDrive/Big-Data-Derb/Input/nyra_tracking_table.csv  


In [17]:
nyra_tracking = pd.read_csv(os.path.join(INPUT,"nyra_tracking_table.csv"))
nyra_start = pd.read_csv(os.path.join(INPUT,"nyra_start_table.csv"))
nyra_race = pd.read_csv(os.path.join(INPUT,"nyra_race_table.csv"))
nyra_2019 = pd.read_csv(os.path.join(INPUT,"nyra_2019_complete.csv"))

このコンペティションの目的は、競馬の戦術、ドラフト戦略、経路効率などを分析することです。このコンペティションでは、未発表の座標データ、および基本的なレース情報を使って、モデルを作成します。

あなたの研究は、馬主、調教師、獣医師が、馬のパフォーマンスと福祉の関係をよりよく理解するために役立ちます。データ分析がうまくいけば、馬の福祉が大きく向上する可能性があります。

現代のアスリートにとって、ケガの予防は非常に重要な要素です。競馬のような動物が関わるスポーツも、人間のスポーツと何ら変わりはない。一般的に、動作の効率化はパフォーマンスの向上と怪我の予防の両方に相関している。

現在では、心拍数、心電図、縦方向の動き、背骨・腹骨の動き、内側・外側偏位、トータルパワー、トータル着地振動の測定など、豊富なデータが収集されています。何が最も良い影響を与えるのかを読み解くために、あなたのデータサイエンスのスキルと分析が必要なのです。

このコンペティションでは、この新しいデータの1つの側面を解釈するモデルを作成します。あなたは、レース中の馬のX/Y座標マッピングにいち早くアクセスすることができます。このデータを使って、騎手の意思決定の分析、競走路の比較、ドラフティングの相対的な重要性の測定などを行うことができるかもしれません。豊富なデータで、クリエイティブな問題解決力を発揮してください。

ニューヨーク競馬協会（NYRA）とニューヨーク・サラブレッド馬主協会（NYTHA）は、アケダクト競馬場、ベルモントパーク、サラトガ競馬場で世界クラスのサラブレッドレースを開催しています。

皆様のご協力により、NYRAとNYTHAは膨大なデータをより深く理解することができ、伝統のある競馬業界において新しいレースや調教のあり方につながる可能性があります。馬の追跡データの利用が改善されれば、馬の福祉、パフォーマンス、騎手の意思決定の向上に貢献できるかもしれません。

In [20]:
nyra_2019.head()

,track_id,race_date,race_number,program_number,trakus_index,latitude,longitude,distance_id,course_type,track_condition,run_up_distance,race_type,purse,post_time,weight_carried,jockey,odds
0,AQU,2019-01-01,9,6,72,40.672902,-73.827607,600,D,GD,48,CLM,25000,420,120,Andre Shivnarine Worrie,2090
1,AQU,2019-01-01,9,6,73,40.672946,-73.827587,600,D,GD,48,CLM,25000,420,120,Andre Shivnarine Worrie,2090
2,AQU,2019-01-01,9,6,74,40.672990,-73.827568,600,D,GD,48,CLM,25000,420,120,Andre Shivnarine Worrie,2090
3,AQU,2019-01-01,9,6,63,40.672510,-73.827781,600,D,GD,48,CLM,25000,420,120,Andre Shivnarine Worrie,2090
4,AQU,2019-01-01,9,6,64,40.672553,-73.827762,600,D,GD,48,CLM,25000,420,120,Andre Shivnarine Worrie,2090


In [21]:
nyra_2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5228430 entries, 0 to 5228429
Data columns (total 17 columns):
 #   Column           Dtype  
---  ------           -----  
 0   track_id         object 
 1   race_date        object 
 2   race_number      int64  
 3   program_number   object 
 4   trakus_index     int64  
 5   latitude         float64
 6   longitude        float64
 7   distance_id      int64  
 8   course_type      object 
 9   track_condition  object 
 10  run_up_distance  int64  
 11  race_type        object 
 12  purse            int64  
 13  post_time        int64  
 14  weight_carried   int64  
 15  jockey           object 
 16  odds             int64  
dtypes: float64(2), int64(8), object(7)
memory usage: 678.1+ MB


| Column          | Dtype    | Description                                                                                                                                                                               | 
| --------------- | -------- | ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | 
| track_id        | object   |                                                                                                                                                                                           | 
|  race_date      | object   |                                                                                                                                                                                           | 
| race_number     | int64    |                                                                                                                                                                                           | 
| program_number  | object   |                                                                                                                                                                                           | 
| trakus_index    | int64    | Trakusは、サラブレッド競馬のスピード・距離曲線計測システムで、レース中の各馬の移動距離に対するスピードを計測し、そこからPOC（ポイントオブコール）時の馬体間距離を計測することができます。 | 
| latitude        | float64  |                                                                                                                                                                                           | 
| longitude       |  float64 |                                                                                                                                                                                           | 
| distance_id     | int64    |                                                                                                                                                                                           | 
| course_type     | object   |                                                                                                                                                                                           | 
| track_condition | object   |                                                                                                                                                                                           | 
| run_up_distance | int64    | "Run-up "とは、ゲートが置かれレースのタイミングが始まるまでの距離、つまり馬がレースの公表距離に到達した時点のことである。                                                                 | 
| race_type       | object   |                                                                                                                                                                                           | 
| purse           |  int64   | 競馬では、一定期間に特定の競馬場で行われるレースの馬主に支払われる賞金の総額、またはレースの総賞金のうち上位入賞馬に与えられる割合のことを指す。                                          | 
|  post_time      | int64    | ポストタイムとは、競馬の指定された開始時刻のことです。                                                                                                                                    | 
| weight_carried  | int64    |                                                                                                                                                                                           | 
| jockey          | object   |                                                                                                                                                                                           | 
| odds            | int64    |                                                                                                                                                                                           | 

In [22]:
nyra_2019.describe().style.background_gradient(cmap = "Purples")

,race_number,trakus_index,latitude,longitude,distance_id,run_up_distance,purse,post_time,weight_carried,odds
count,5228430.000000,5228430.000000,5228430.000000,5228430.000000,5228430.000000,5228430.000000,5228430.000000,5228430.000000,5228430.000000,5228430.000000
mean,5.505409,183.865213,41.203867,-73.777024,756.315127,67.919189,88878.907779,420.424807,120.713291,1482.919877
std,2.860656,118.332690,0.977123,0.047111,179.688689,38.922712,127824.025516,280.254500,4.351881,1952.415229
min,1.000000,1.000000,40.666711,-73.832601,450.000000,0.000000,16000.000000,100.000000,110.000000,0.000000
25%,3.000000,88.000000,40.673456,-73.828834,600.000000,45.000000,41000.000000,233.000000,118.000000,335.000000
50%,6.000000,176.000000,40.713934,-73.769148,800.000000,54.000000,62000.000000,349.000000,120.000000,730.000000
75%,8.000000,264.000000,40.717393,-73.725636,850.000000,80.000000,80000.000000,516.000000,122.000000,1740.000000
max,13.000000,1062.000000,43.073992,-73.714826,2000.000000,280.000000,1500000.000000,1259.000000,160.000000,19100.000000


In [29]:
nyra_tracking.track_id.value_counts()

AQU    2158369
BEL    1947134
SAR    1122927
Name: track_id, dtype: int64

In [30]:
nyra_tracking.race_number.value_counts()

8     582413
6     575217
5     559884
7     554623
4     523846
2     515115
1     508808
9     492915
3     490256
10    283141
11     98804
12     31725
13     11683
Name: race_number, dtype: int64

In [31]:
nyra_tracking.program_number.value_counts()

4      611396
3      611025
5      606666
2      602597
1      599979
6      579893
7      486905
8      373202
9      266526
10     186835
11     117532
12      71363
1A      56753
13      26507
14      15081
15       7917
2B       4081
16       3591
3X        334
1X        247
Name: program_number, dtype: int64

In [32]:
nyra_tracking['race_date'] = pd.to_datetime(nyra_tracking['race_date'])
nyra_tracking['day'], nyra_tracking['month'] = nyra_tracking['race_date'].dt.day, nyra_tracking['race_date'].dt.month